In [1]:
import pickle
import pandas as pd
import deepchem as dc
import numpy as np
from rdkit import Chem

import warnings
warnings.filterwarnings('ignore')

Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
with open('xgb_maccs_model.pkl', 'rb') as f:
    xgb_maccs_model = pickle.load(f)

In [17]:
data_pre = pd.read_excel('../data/validation.xlsx')
data = pd.read_csv('../data/BST.csv',encoding = 'gb2312')

In [18]:
data_pre

,SMILES,Name,Labels
0,c1occc1[C@H](OC(=O)[C@H]([C@]234)O4)[C@]3(C)C[...,Zapoterin,Bitter
1,CC(=O)O[C@@H]1[C@@H](OC)[C@H](C)[C@@H]([C@@]2(...,Picrasin C,Bitter
2,c1occc1[C@H](OC(=O)[C@H]([C@]234)O4)[C@]3(C)CC...,Obacunone,Bitter
3,C[C@]12[C@@H]3[C@]4(C)[C@H]([C@](C)(O)[C@@H]([...,Nigakihemiacetal A,Bitter
4,C[C@]12[C@@H]3[C@]4(C)[C@H](C(C)=C(C3=O)OC)C[C...,Neoquassin,Bitter
...,...,...,...
531,COC1CC(OC(C1OC1CC(OC)C(C(O1)C)OC1CC(OC)C(C(O1)...,NaN,Sweet
532,CC=Cc1ccc(cc1)OC,NaN,Sweet
533,O=CC=Cc1ccccc1,NaN,Sweet
534,CC1(C2CCC3(C(C2(CCC1OC4C(C(C(C(O4)C(=O)[O-])O)...,NaN,Sweet


In [19]:
def valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None


invalid_smiles = data_pre[~data_pre['SMILES'].apply(valid_smiles)]
print("无效的 SMILES 条目：")
print(invalid_smiles)

无效的 SMILES 条目：
Empty DataFrame
Columns: [SMILES, Name, Labels]
Index: []


In [20]:
featurizer_maccs = dc.feat.MACCSKeysFingerprint()
features = featurizer_maccs.featurize(data['SMILES'])
pred_features = featurizer_maccs.featurize(data_pre['SMILES'])

[20:22:52] WARNING: not removing hydrogen atom without neighbors


In [21]:
from sklearn.feature_selection import VarianceThreshold


vt = VarianceThreshold(threshold = (.98 * (1 - .98)))


data_maccs_new = vt.fit_transform(features)
data_maccs_mask = vt.get_support(indices=True)
pred_maccs_features = pred_features[:, data_maccs_mask]

In [22]:
pred_maccs_features.shape

(536, 135)

In [23]:
pred_pro = xgb_maccs_model.predict_proba(pred_maccs_features)
pred = xgb_maccs_model.predict(pred_maccs_features)

In [24]:
pred

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [25]:
pred_pro.tolist()

[[0.993890643119812, 0.00012049177894368768, 0.006903546862304211],
 [0.9972737431526184, 0.0029517903458327055, 0.0010460916673764586],
 [0.9980428218841553, 0.00031763079459778965, 0.0009648017003200948],
 [0.9946840405464172, 0.0013188752345740795, 0.0024175948929041624],
 [0.9969268441200256, 0.0004735373368021101, 0.0031416681595146656],
 [0.979477047920227, 0.00028489905525930226, 0.014114654622972012],
 [0.9890087246894836, 0.0004307707422412932, 0.008926532231271267],
 [0.9890087246894836, 0.0004307707422412932, 0.008926532231271267],
 [0.9866304993629456, 0.00024855596711859107, 0.045840270817279816],
 [0.9921848177909851, 0.002010658849030733, 0.006864381954073906],
 [0.9988743662834167, 0.0001772524556145072, 0.0014628105564042926],
 [0.980553388595581, 0.00010390472743893042, 0.02033771574497223],
 [0.952172040939331, 0.0013376978458836675, 0.03850517049431801],
 [0.7167387008666992, 0.0003186799876857549, 0.2917470335960388],
 [0.9527869820594788, 0.00022865270148031414, 0

In [26]:
result_list = []

for item in pred:
    if np.array_equal(item, [1, 0, 0]):
        result_list.append("Bitter")
    elif np.array_equal(item, [0, 1, 0]):
        result_list.append("Sweet")
    elif np.array_equal(item, [0, 0, 1]):
        result_list.append("Tasteless")
    elif np.array_equal(item, [1, 1, 0]):
        result_list.append("Bittersweet")
    elif np.array_equal(item, [1, 0, 1]):
        result_list.append("Bitterless")
    elif np.array_equal(item, [0, 1, 1]):
        result_list.append("Sweetless")
    elif np.array_equal(item, [0, 0, 0]):
        result_list.append("Low confidence")
    elif np.array_equal(item, [1, 1, 1]):
        result_list.append("Low confidence")
print(result_list)

['Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Sweet', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Sweet', 'Sweet', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bittersweet', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter', 'Bitter

In [27]:
len(result_list)

536

In [28]:
df = pd.DataFrame({
    'SMILES':data_pre['SMILES'],
#     'InDomain':data_pre['InDomain'],
    'pred':pred_pro.tolist(),
    'class':result_list
})

In [29]:
df

,SMILES,pred,class
0,c1occc1[C@H](OC(=O)[C@H]([C@]234)O4)[C@]3(C)C[...,"[0.993890643119812, 0.00012049177894368768, 0....",Bitter
1,CC(=O)O[C@@H]1[C@@H](OC)[C@H](C)[C@@H]([C@@]2(...,"[0.9972737431526184, 0.0029517903458327055, 0....",Bitter
2,c1occc1[C@H](OC(=O)[C@H]([C@]234)O4)[C@]3(C)CC...,"[0.9980428218841553, 0.00031763079459778965, 0...",Bitter
3,C[C@]12[C@@H]3[C@]4(C)[C@H]([C@](C)(O)[C@@H]([...,"[0.9946840405464172, 0.0013188752345740795, 0....",Bitter
4,C[C@]12[C@@H]3[C@]4(C)[C@H](C(C)=C(C3=O)OC)C[C...,"[0.9969268441200256, 0.0004735373368021101, 0....",Bitter
...,...,...,...
531,COC1CC(OC(C1OC1CC(OC)C(C(O1)C)OC1CC(OC)C(C(O1)...,"[0.0049570659175515175, 0.9480160474777222, 0....",Sweet
532,CC=Cc1ccc(cc1)OC,"[0.018386172130703926, 0.9843757748603821, 0.0...",Sweet
533,O=CC=Cc1ccccc1,"[0.909917414188385, 0.05744956433773041, 0.026...",Bitter
534,CC1(C2CCC3(C(C2(CCC1OC4C(C(C(C(O4)C(=O)[O-])O)...,"[0.04047691822052002, 0.9342703819274902, 0.00...",Sweet


In [30]:
df.to_csv('result_validation.csv')

# Single-Molecule Prediction

In [94]:
food_features = featurizer_maccs.featurize('C[C@H]1[C@@H]([C@H]([C@H]([C@@H](O1)O[C@@H]2[C@H]([C@@H]([C@H](O[C@H]2OC3=CC(=C4C(=O)C[C@H](OC4=C3)C5=CC=C(C=C5)O)O)CO)O)O)O)O)O')

In [95]:
food_features.shape

(1, 167)

In [96]:
food_maccs_features = food_features[:, data_maccs_mask]

In [97]:
food_maccs_features.shape

(1, 134)

In [98]:
pred = xgb_maccs_model.predict_proba(food_maccs_features)
rounded_array = np.around(pred, decimals=2)

In [99]:
rounded_array

array([[0.78, 0.12, 0.1 ]], dtype=float32)

In [5]:
data_pre = 'C[C@H]1[C@@H]([C@H]([C@H]([C@@H](O1)O[C@@H]2[C@H]([C@@H]([C@H](O[C@H]2OC3=CC(=C4C(=O)C[C@H](OC4=C3)C5=CC=C(C=C5)O)O)CO)O)O)O)O)O'
data = pd.read_csv('../data/BST.csv', encoding='gb2312')
featurizer_maccs = dc.feat.MACCSKeysFingerprint()
features = featurizer_maccs.featurize(data['SMILES'])
food_features = featurizer_maccs.featurize(data_pre)

In [7]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=(.98 * (1 - .98)))
data_maccs_new = vt.fit_transform(features)
data_maccs_mask = vt.get_support(indices=True)
food_maccs_features = food_features[:, data_maccs_mask]

In [8]:
food_maccs_features

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0]])

In [9]:
pred = xgb_maccs_model.predict(food_maccs_features)

In [10]:
pred

array([[1, 0, 0]])

In [12]:
result_list = pred.flatten().tolist()

In [16]:
result_list = pred.flatten().tolist()

# Map prediction to taste
if result_list == [1, 0, 0]:
    taste = "Bitter"
elif result_list == [0, 1, 0]:
    taste = "Sweet"
elif result_list == [0, 0, 1]:
    taste = "Tasteless"
else:
    taste = "Unknown"

In [18]:
taste

'苦味'